# Connecting to Amazon Redshift

# Overview
`redshift_connector` provides multiple options when it comes to establishing a connection to an Amazon Redshift cluster. These options are discussed and shown below.

## Using Database credentials
Raw database credentials can be used for establishing a connection to an Amazon Redshift cluster. While straight forward, this approach lack the strong security and user access controls provides by Identity and access management (IAM) and identity provider (IdP) plugins.

In [ ]:
import redshift_connector

# establish a connection to an Amazon Redshift cluster

# here we use "with" statements to ensure connection
# and cursor resources are cleaned up once we are finished
# with them

with redshift_connector.connect(
    host='examplecluster.abc123xyz789.us-west-1.redshift.amazonaws.com',
    database='dev',
    user='awsuser',
    password='my_password'
    # port value of 5439 is specified by default
) as conn:
    with conn.cursor() as cursor:
       # Please note: autocommit is disabled by default, per DB-API specification
       # If you'd like to commit your changes, manually commit or enable autocommit
       # on the cursor object

       # conn.commit()  # manually commits
       # conn.autocommit = True  # enables autocommit for subsequent SQL statements
        cursor.execute("create table book(bookname varchar,author varchar)")
        cursor.executemany("insert into book (bookname, author) values (%s, %s)",
                        [
                            ('One Hundred Years of Solitude', 'Gabriel García Márquez'),
                            ('A Brief History of Time', 'Stephen Hawking')
                        ]
                      )
        cursor.execute("select * from book")

        result: tuple = cursor.fetchall()
        print(result)

## Using IAM Credentials
IAM Credentials can be supplied directly to ``connect(...)`` using an AWS profile. This approach allows users the option of using temporary credentials and limiting the permissions the connected user has.

In [ ]:
import redshift_connector

# Connects to Redshift cluster using IAM credentials from default profile defined in ~/.aws/credentials
conn: redshift_connector.Connection = redshift_connector.connect(
    iam=True,
    database='dev',
    db_user='awsuser',
    password='',
    user='',
    cluster_identifier='examplecluster',
    profile='default'
 )

Below a sample `~/.aws/config` and `~/.aws/credentials` are shown. Please note that `redshift_connector` requires a `region` to be specified when usign IAM authentication. The region can be specified either in `~/.aws/config` or passed directly to `redshift_connector.connect(...)`. In the case where a region is specified in both `~/.aws/config` and `redshift_connector.connect(...)`, the value provided to `redshift_connector.connect(...)` will be used.

`~/.aws/credentials`
```
[default]
aws_access_key_id="my_aws_access_key_id"
aws_secret_access_key="my_aws_secret_access_key"
aws_session_token="my_aws_session_token"
```
`~/.aws/config`
```
[default]
region=us-west-2
```

Alternatively, IAM credentials can be supplied directly to ``connect(...)`` using AWS credentials as shown below:

In [ ]:
import redshift_connector

# Connects to Redshift cluster using IAM credentials from default profile defined in ~/.aws/credentials
conn: redshift_connector.Connection = redshift_connector.connect(
    iam=True,
    database='dev',
    db_user='awsuser',
    password='',
    user='',
    cluster_identifier='examplecluster',
    access_key_id="my_aws_access_key_id",
    secret_access_key="my_aws_secret_access_key",
    session_token="my_aws_session_token",
    region="us-east-2"
 )


# Connecting using an Amazon Redshift Authentication Profile
An Amazon Redshift authentication profile can be used for authentication with Amazon Redshift via ``redshift_connector``. This approach allows connection properties to be stored in the server side and retrieved by ``redshift_connector``. Any connection parameter which appears in both the authentication profile and is directly provided to ``redshift_connector.connect(...)`` will be overriden by the value provided in the authentication profile.

Please see the Amazon Redshift documentation to learn how to create and delete authentication profiles.

In the following example we will be creating, using, and deleting an authentication profile. For this use case we would like to connect to an Amazon Redshift cluster, but store no credential or cluster information within the Python script. This will improve the portability of our code as well as its security.

Firstly, we will create the Amazon Redshift authentication profile by using ``boto3``.

In [ ]:
import boto3
from botocore.exceptions import ClientError
import json


authentication_profile_contents = {
    'host': 'examplecluster.abc123xyz789.us-west-1.redshift.amazonaws.com',
    'region': 'us-west-1',
    'cluster_identifier': 'examplecluster',
    'db_name': 'dev'
}

try:
    client = boto3.client("redshift")
    client.create_authentication_profile(
        AuthenticationProfileName="QAProfile",
        AuthenticationProfileContent=json.dumps(authentication_profile_contents)
    )
except ClientError:
    raise

The Redshift authentication profile, named ``QAProfile`` has been created. This profile is intended for use by a QA team who would like to avoid hard-coded references to a specific cluster in their projects. Its contents are in JSON format and contain fields such as ``host`` and ``cluster_identifier``.

Next we will establish a connection to this cluster by using this authentication profile.

In [ ]:
import redshift_connector
import os

with redshift_connector.connect(
    iam=True,
    region='us-west-2',
    access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    session_token=os.environ["AWS_SESSION_TOKEN"],
    auth_profile="QAProfile",
    db_user="bobby_tables"
) as conn:
    pass

Noting the ``region`` parameter above, we can see that while the Amazon Redshift authentication profile lives in ``us-west-2``, ``examplecluster`` lives in ``us-west-1``. When retrieving temporary IAM credentials to connect to this cluster, the ``region`` provided in the authentication profile will be used.

Please see the ``redshift_connector.RedshiftProperty`` class for guidance on how to define the key and value contents of the JSON authentication profile contents.

Finally, we will delete this authentication profile for demonstration purposes.


In [ ]:
try:
    client = boto3.client("redshift")
    client.delete_authentication_profile(
        AuthenticationProfileName="QAProfile",
    )
except ClientError:
    raise

# Connecting using Identity Provider (IdP) Plugins
Please refer to the following [Amazon Redshift documentation](https://docs.aws.amazon.com/redshift/latest/mgmt/options-for-providing-iam-credentials.html) for instructions on configuring the desired IdP.
Check out our blog post on [AWS Big Data Blog](https://aws.amazon.com/blogs/big-data/federated-api-access-to-amazon-redshift-using-an-amazon-redshift-connector-for-python/) to learn more about integration with Okta IdP.

## Authenticating using Active Directory Federation Service (ADFS) identity provider plugin

In [ ]:
import redshift_connector

conn: redshift_connector.Connection = redshift_connector.connect(
    iam=True,
    database='dev',
    cluster_identifier='my-testing-cluster',
    credentials_provider='AdfsCredentialsProvider',
    user='brooke@myadfshostname.com',
    password='Hunter2',
    idp_host='myadfshostname.com'
)



## Authenticating using Azure identity provider plugin
Values for `client_id`, `client_secret` can be created and found  within the Enterprise Application created with Azure.

In [ ]:
import redshift_connector

conn: redshift_connector.Connection = redshift_connector.connect(
    iam=True,
    database='dev',
    cluster_identifier='my-testing-cluster',
    credentials_provider='AzureCredentialsProvider',
    user='brooke@myazure.org',
    password='Hunter2',
    idp_tenant='my_idp_tenant',
    client_id='my_client_id',
    client_secret='my_client_secret',
    preferred_role='arn:aws:iam:123:role/MyFirstDinnerRoll'
)

## Authenticating using Azure Browser identity provider plugin

In [ ]:
import redshift_connector

conn: redshift_connector.Connection = redshift_connector.connect(
    iam=True,
    database='dev',
    cluster_identifier='my-testing-cluster',
    credentials_provider='BrowserAzureCredentialsProvider',
    user='brooke@myazure.org',
    password='',
    idp_tenant='my_idp_tenant',
    client_id='my_client_id',
    client_secret='my_client_secret',
)

## Authenticating using Okta identity provider plugin
Values for `idp_host`, `app_id`, and `app_name` can be located within the Okta application created.

In [ ]:
import redshift_connector

conn: redshift_connector.Connection = redshift_connector.connect(
    iam=True,
    database='dev',
    cluster_identifier='my-testing-cluster',
    credentials_provider='OktaCredentialsProvider',
    user='brooke@myazure.org',
    password='hunter2',
    idp_host='my_idp_host',
    app_id='my_first_appetizer',
    app_name='dinner_party'
)

## Authenticating using JumpCloud via generic Saml Browser identity provider plugin

In [ ]:
import redshift_connector

conn: redshift_connector.Connection = redshift_connector.connect(
    iam=True,
    database='dev',
    cluster_identifier='my-testing-cluster',
    credentials_provider='BrowserSamlCredentialsProvider',
    user='brooke@myjumpcloud.org',
    password='',
    login_url='https://sso.jumpcloud.com/saml2/plustwo_melody'
)